In [1]:
import time
from joblib import Parallel, delayed
import xarray as xr
import pandas as pd
import scipy as sp
import numpy as np
from pathlib import Path
from os import listdir, makedirs, remove
from os.path import isfile, join, exists
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib
from cartopy import crs
from sklearn import metrics

font = {'weight' : 'bold', 'size' : 14}
matplotlib.rc('font', **font)
# show all rows
pd.set_option('display.max_rows', None)
# # hide warnings
import warnings
warnings.filterwarnings('ignore')

# 1. Define functions

In [2]:
def filter_nan(s,o):
    """
    this functions removed the data  from simulated and observed data
    whereever the observed data contains nan
    s: simulated
    o: observed
    """
    if np.sum(~np.isnan(s*o))>=1:
#         data = np.array([s.flatten(),o.flatten()])
        data = np.array([s,o])
        data = np.transpose(data)
        data = data[~np.isnan(data).any(1)]
        s = data[:,0]
        o = data[:,1]
    return s, o

def correlation(s,o):
    """
    correlation coefficient
        s: simulated
        o: observed
        correlation: correlation coefficient
    """
    s,o = filter_nan(s,o)
    if s.size == 0:
        corr = np.NaN
    else:
        corr = np.corrcoef(o, s)[0,1]
        
    return corr

def rmse(s,o):
    """
    Root Mean Squared Error
    s: simulated
    o: observed
    rmses: root mean squared error
    """
    s,o = filter_nan(s,o)
    return np.sqrt(np.mean((s-o)**2))

def mae(s,o):
    """
    Mean Absolute Error
    s: simulated
    o: observed
    """
    s,o = filter_nan(s,o)
    return np.mean(abs(s-o))

def bias(s,o):
    """
    Bias
    s: simulated
    o: observed
    """
    s,o = filter_nan(s,o)
    return np.mean(s-o)

def index_agreement(s, o):
    """index of agreement Willmott (1981, 1982) 
        s: simulated
        o: observed
        ia: index of agreement
    """
    s,o = filter_nan(s,o)
    ia = 1 -(np.sum((o-s)**2))/(np.sum((np.abs(s-np.mean(o))+np.abs(o-np.mean(o)))**2))
    return ia

def rdmf(s, o):
    """Relative Difference of the mean footprint"""
    s,o = filter_nan(s,o)
    # zero division, e.g. both footprints are zero,set footprint 1
    s.setflags(write=1)
    o.setflags(write=1)
    z = s+o
    s[z==0]=1
    o[z==0]=1
#     percentage = np.mean(s - o)/np.mean((s+o)/2 )* 100
    percentage = (s - o)/o * 100
    
    return np.mean(percentage),np.std(percentage)

def get_darray_r(array_r,darray_temp):
    """convert array r to xarray dataarray
        array_r: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_r = xr.DataArray(
        array_r,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Pearson correlation coefficient"
        )
    )
    darray_r.name = "r"
    darray_r.attrs["units"] = "1"
    darray_r.attrs["long_name"] = "Pearson correlation coefficient"

    darray_r.lon.attrs['units'] = "degrees_east"
    darray_r.lon.attrs['long_name'] = "longitude in degree east"
    darray_r.lon.attrs['description'] = "grid cell centers"
    darray_r.lon.attrs['axis'] = "X"

    darray_r.lat.attrs['units'] = "degrees_north"
    darray_r.lat.attrs['long_name'] = "longitude in degree north"
    darray_r.lat.attrs['description'] = "grid cell centers"
    darray_r.lat.attrs['axis'] = "Y"
    return darray_r

def get_darray_rmse(array_rmse,darray_temp):
    """convert array rmse to xarray dataarray
        darray_rmse: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_rmse = xr.DataArray(
        array_rmse,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Root Mean Squared Error"
        )
    )
    darray_rmse.name = "rmse"
    darray_rmse.attrs["units"] = "ppm per (micromol m-2 s-1)"
    darray_rmse.attrs["long_name"] = "Root Mean Squared Error"

    darray_rmse.lon.attrs['units'] = "degrees_east"
    darray_rmse.lon.attrs['long_name'] = "longitude in degree east"
    darray_rmse.lon.attrs['description'] = "grid cell centers"

    darray_rmse.lat.attrs['units'] = "degrees_north"
    darray_rmse.lat.attrs['long_name'] = "longitude in degree north"
    darray_rmse.lat.attrs['description'] = "grid cell centers"
    return darray_rmse

def get_darray_mae(array_mae,darray_temp):
    """convert array mae to xarray dataarray
        darray_mae: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_mae = xr.DataArray(
        array_mae,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Mean Absolute Error"
        )
    )
    darray_mae.name = "mae"
    darray_mae.attrs["units"] = "ppm per (micromol m-2 s-1)"
    darray_mae.attrs["long_name"] = "Mean Absolute Error"

    darray_mae.lon.attrs['units'] = "degrees_east"
    darray_mae.lon.attrs['long_name'] = "longitude in degree east"
    darray_mae.lon.attrs['description'] = "grid cell centers"

    darray_mae.lat.attrs['units'] = "degrees_north"
    darray_mae.lat.attrs['long_name'] = "longitude in degree north"
    darray_mae.lat.attrs['description'] = "grid cell centers"
    return darray_mae

def get_darray_bias(array_bias,darray_temp):
    """convert array bias to xarray dataarray
        darray_bias: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_bias = xr.DataArray(
        array_bias,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Mean Error"
        )
    )
    darray_bias.name = "bias"
    darray_bias.attrs["units"] = "ppm per (micromol m-2 s-1)"
    darray_bias.attrs["long_name"] = "Mean Error"

    darray_bias.lon.attrs['units'] = "degrees_east"
    darray_bias.lon.attrs['long_name'] = "longitude in degree east"
    darray_bias.lon.attrs['description'] = "grid cell centers"

    darray_bias.lat.attrs['units'] = "degrees_north"
    darray_bias.lat.attrs['long_name'] = "longitude in degree north"
    darray_bias.lat.attrs['description'] = "grid cell centers"
    return darray_bias

def get_darray_IoA(array_IoA,darray_temp):
    """convert array IoA to xarray dataarray
        darray_IoA: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_IoA = xr.DataArray(
        array_IoA,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Index of agreement"
        )
    )
    darray_IoA.name = "IoA"
    darray_IoA.attrs["units"] = "1"
    darray_IoA.attrs["long_name"] = "Index of agreement"

    darray_IoA.lon.attrs['units'] = "degrees_east"
    darray_IoA.lon.attrs['long_name'] = "longitude in degree east"
    darray_IoA.lon.attrs['description'] = "grid cell centers"

    darray_IoA.lat.attrs['units'] = "degrees_north"
    darray_IoA.lat.attrs['long_name'] = "longitude in degree north"
    darray_IoA.lat.attrs['description'] = "grid cell centers"
    return darray_IoA

def get_darray_rpd(array_rpd,darray_temp):
    """convert array rpd to xarray dataarray
        darray_rpd: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_rpd = xr.DataArray(
        array_rpd,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Relative percentage difference"
        )
    )
    darray_rpd.name = "rpd"
    darray_rpd.attrs["units"] = "%"
    darray_rpd.attrs["long_name"] = "Relative percentage difference"

    darray_rpd.lon.attrs['units'] = "degrees_east"
    darray_rpd.lon.attrs['long_name'] = "longitude in degree east"
    darray_rpd.lon.attrs['description'] = "grid cell centers"
    darray_rpd.lon.attrs['axis'] = "X"

    darray_rpd.lat.attrs['units'] = "degrees_north"
    darray_rpd.lat.attrs['long_name'] = "longitude in degree north"
    darray_rpd.lat.attrs['description'] = "grid cell centers"
    darray_rpd.lat.attrs['axis'] = "Y"
    return darray_rpd

def get_darray_rdmf(array_rdmf,darray_temp):
    """convert array rdmf to xarray dataarray
        darray_rdmf: target array
        darray_temp: provide darray shape, i.e. lon and lat for the array
    """
    darray_rdmf = xr.DataArray(
        array_rdmf,
        dims=["lat","lon"],
        coords=dict(
            lat = np.array(darray_temp['lat']),
            lon = np.array(darray_temp['lon'])
        ),
        attrs = dict(
            description = "Relative difference of the mean footprint"
        )
    )
    darray_rdmf.name = "rdmf"
    darray_rdmf.attrs["units"] = "%"
    darray_rdmf.attrs["long_name"] = "Relative difference of the mean footprint"

    darray_rdmf.lon.attrs['units'] = "degrees_east"
    darray_rdmf.lon.attrs['long_name'] = "longitude in degree east"
    darray_rdmf.lon.attrs['description'] = "grid cell centers"
    darray_rdmf.lon.attrs['axis'] = "X"

    darray_rdmf.lat.attrs['units'] = "degrees_north"
    darray_rdmf.lat.attrs['long_name'] = "longitude in degree north"
    darray_rdmf.lat.attrs['description'] = "grid cell centers"
    darray_rdmf.lat.attrs['axis'] = "Y"
    return darray_rdmf

def get_6stat(darray_afternoon_stilt,darray_afternoon_flexpart):
    """ compute 6 statistics, r, rmse, mae, bias, IoA, rpd
        darray_afternoon_stilt: data array with the same shape
        darray_afternoon_flexpart: data array with the same shape
    """
    nlon = len(darray_afternoon_stilt['lon'])
    nlat = len(darray_afternoon_stilt['lat'])
    array_r = np.empty((nlat,nlon))
    array_r[:] = np.NaN
    array_rmse = np.empty((nlat,nlon))
    array_rmse[:] = np.NaN
    array_mae = np.empty((nlat,nlon))
    array_mae[:] = np.NaN
    array_bias = np.empty((nlat,nlon))
    array_bias[:] = np.NaN
    array_IoA = np.empty((nlat,nlon))
    array_IoA[:] = np.NaN
    array_rpd = np.empty((nlat,nlon))
    array_rpd[:] = np.NaN
    for lon in range(nlon):
        for lat in range(nlat):
            array_r[lat,lon] = correlation(darray_afternoon_stilt.values[:,lat,lon],darray_afternoon_flexpart.values[:,lat,lon])
            array_rmse[lat,lon] = rmse(darray_afternoon_stilt.values[:,lat,lon],darray_afternoon_flexpart.values[:,lat,lon])
            array_mae[lat,lon] = mae(darray_afternoon_stilt.values[:,lat,lon],darray_afternoon_flexpart.values[:,lat,lon])
            array_bias[lat,lon] = bias(darray_afternoon_stilt.values[:,lat,lon],darray_afternoon_flexpart.values[:,lat,lon])
            array_IoA[lat,lon] = index_agreement(darray_afternoon_stilt.values[:,lat,lon],darray_afternoon_flexpart.values[:,lat,lon])
            array_rpd[lat,lon] = rpd(darray_afternoon_stilt.values[:,lat,lon],darray_afternoon_flexpart.values[:,lat,lon])
        if lon % 20 ==0:
            print(lon,end="..")
    print("done")
    return array_r, array_rmse, array_mae, array_bias, array_IoA, array_rpd

def plot_conf_matrix(df_flexpart, df_stilt, yearmon, lt, threshold, ax):
    y_flexpart = df_flexpart>threshold
    y_stilt = df_stilt>threshold
    conf_matrix = metrics.confusion_matrix(y_stilt, y_flexpart)
    accuracy = round(np.sum(conf_matrix.diagonal())/np.sum(conf_matrix)*100,1)

    alpha = [f'<={threshold}',f'>{threshold}']
    ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.6)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
    xaxis = np.arange(len(alpha))
    ax.set_xticks(xaxis)
    ax.set_yticks(xaxis)
    ax.set_xticklabels(alpha)
    ax.set_yticklabels(alpha);
    ax.set_xlabel('Flexpart ∆14C', fontsize=12)
    ax.set_ylabel('Stilt ∆14C', fontsize=12)
    ax.set_title(f'{yearmon} {lt}LT, agreement={accuracy}%', fontsize=12, loc = "left")

def plot_hist(df_flexpart, df_stilt, yearmon,  ax):
    data1 = df_flexpart['mean']
    data2 = df_stilt['mean']

    nbin = [i*0.2 for i in range(50)]
    ax.hist(data1, bins=nbin, alpha=0.7, label='Flexpart afternoon ∆14C')
    ax.hist(data2, bins=nbin, alpha=0.7, label='Stilt afternoon ∆14C')

    ax.grid()
    ax.set_xlim(0,10)
    ax.set_xticks(range(11))
    ax.set_xticklabels(range(11))
    ax.set_xlabel('∆14C, [‰]')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Histogram Comparison, {yearmon}',loc = "left")
    ax.tick_params(axis='x', labelrotation = 0)
    ax.legend()

In [3]:
def merge_footprint(path, StationID, sdate, edate, fname="foot_nest", domain="eu"):
    """
    Merge hourly footprint files for a station over a period and save as NetCDF.

    Parameters:
        path (str): Base directory for FLEXPART output.
        StationID (str): Station identifier.
        sdate (str): Start date, format 'YYYY-MM-DD'.
        edate (str): End date, format 'YYYY-MM-DD'.
        fname (str): File name to read ('foot' or 'foot_nest').
        domain (str): Domain identifier for output file naming.
    """
    start_time = time.time()
    StationID = StationID.upper()
    period = pd.date_range(start=sdate, end=edate)
    darray_flexpart = None
    i = 0
    print(f"{StationID}: ", flush=True)
    for date in period:
        Year = str(date.year)
        Month = str(date.month).zfill(2)
        Day = str(date.day).zfill(2)
        for Hour in range(24):
            folder = f"{Year}x{Month}x{Day}x{str(Hour).zfill(2)}"
            file_path = f"{path}{StationID}/{Year}/{Month}/{folder}/{fname}"
            try:
                darray_temp = xr.open_dataarray(file_path)
                darray_temp = darray_temp.where(darray_temp != 0)
                if darray_flexpart is None:
                    darray_flexpart = darray_temp
                else:
                    darray_flexpart = xr.concat([darray_flexpart, darray_temp], dim="time")
            except Exception as e:
                print(f"Missing or error in file: {file_path} -- {e}")
        i += 1
        if i % 10 == 0:
            print(f"{date}...", end="", flush=True)
    if darray_flexpart is not None:
        file_path_name = f"{path}{StationID}/foot_{domain}_{sdate}_{edate}_flexpart.nc"
        if exists(file_path_name):
            remove(file_path_name)
        darray_flexpart.to_netcdf(file_path_name)
        print(f"\nSaved {file_path_name}")
    else:
        print(f"\nNo data found for {StationID} in period {sdate} to {edate}")
    elapsed_time = time.time() - start_time
    print(f"{StationID} took {elapsed_time:.2f} seconds")

In [8]:
def merge_footprint_out(IN_PATH, OUT_PATH, StationID, sdate, edate, fname="foot_nest", domain="eu"):
    """
    Merge hourly footprint files for a station over a period and save as NetCDF to OUT_PATH.

    Parameters:
        IN_PATH (str): Base directory for FLEXPART input files.
        OUT_PATH (str): Output directory for merged NetCDF.
        StationID (str): Station identifier.
        sdate (str): Start date, format 'YYYY-MM-DD'.
        edate (str): End date, format 'YYYY-MM-DD'.
        fname (str): File name to read ('foot' or 'foot_nest').
        domain (str): Domain identifier for output file naming.
    """
    start_time = time.time()
    StationID = StationID.upper()
    period = pd.date_range(start=sdate, end=edate)
    darray_flexpart = None
    i = 0
    print(f"{StationID}: ", flush=True)
    for date in period:
        Year = str(date.year)
        Month = str(date.month).zfill(2)
        Day = str(date.day).zfill(2)
        for Hour in range(24):
            folder = f"{Year}x{Month}x{Day}x{str(Hour).zfill(2)}"
            file_path = f"{IN_PATH}{StationID}/{Year}/{Month}/{folder}/{fname}"
            try:
                darray_temp = xr.open_dataarray(file_path)
                darray_temp = darray_temp.where(darray_temp != 0)
                if darray_flexpart is None:
                    darray_flexpart = darray_temp
                else:
                    darray_flexpart = xr.concat([darray_flexpart, darray_temp], dim="time")
            except Exception as e:
                print(f"Missing or error in file: {file_path} -- {e}")
        i += 1
        if i % 5 == 0:
            print(f"{date}...", end="", flush=True)
    if darray_flexpart is not None:
        out_dir = join(OUT_PATH, StationID)
        if not exists(out_dir):
            makedirs(out_dir)
        file_path_name = join(out_dir, f"foot_{domain}_{sdate}_{edate}_flexpart.nc")
        if exists(file_path_name):
            remove(file_path_name)
        darray_flexpart.to_netcdf(file_path_name)
        print(f"\nSaved {file_path_name}")
    else:
        print(f"\nNo data found for {StationID} in period {sdate} to {edate}")
    elapsed_time = time.time() - start_time
    print(f"{StationID} took {elapsed_time:.2f} seconds")

In [ ]:
def read_hour_file(file_path):
    try:
        darray_temp = xr.open_dataarray(file_path)
        return darray_temp.where(darray_temp != 0)
    except Exception as e:
        print(f"Missing or error in file: {file_path} -- {e}")
        return None

def merge_footprint_parallel(IN_PATH, OUT_PATH, StationID, sdate, edate, fname="foot_nest", domain="eu", ncpu=5):
    start_time = time.time()
    StationID = StationID.upper()
    period = pd.date_range(start=sdate, end=edate)
    file_paths = []
    print(f"{StationID}: {fname} for {domain} domain", flush=True)
    for date in period:
        Year = str(date.year)
        Month = str(date.month).zfill(2)
        Day = str(date.day).zfill(2)
        for Hour in range(24):
            folder = f"{Year}x{Month}x{Day}x{str(Hour).zfill(2)}"
            file_path = f"{IN_PATH}{StationID}/{Year}/{Month}/{folder}/{fname}"
            file_paths.append(file_path)
    # Parallel read
    results = Parallel(n_jobs=ncpu, verbose=5)(delayed(read_hour_file)(fp) for fp in file_paths)
    # Filter out None
    darrays = [r for r in results if r is not None]
    if darrays:
        darray_flexpart = xr.concat(darrays, dim="time")
        out_dir = join(OUT_PATH, StationID)
        if not exists(out_dir):
            makedirs(out_dir)
        file_path_name = join(out_dir, f"foot_{domain}_{sdate}_{edate}_flexpart.nc")
        if exists(file_path_name):
            remove(file_path_name)
        darray_flexpart.to_netcdf(file_path_name)
        print(f"\nSaved {file_path_name}")
    else:
        print(f"\nNo data found for {StationID} in period {sdate} to {edate}")
    elapsed_time = time.time() - start_time
    print(f"{StationID} took {elapsed_time:.2f} seconds")

# 2. Merge hourly footprint

In [5]:
# Define paths and parameters
lst_Station = ["htm150", "ope120", "gat341", "lin098", "cbw200"]
FLEXPARTv10_PATH = "/data/cupcake/flexpartweb/stations/"
FLEXPARTv11_PATH = "/data/cupcake/flexpartweb/stations/"
FLEXPARTv10_EU_PATH = "/data/cupcake/flexpartweb/stations_eu/"
FLEXPARTv11_EU_PATH = "/data/cupcake/flexpartweb/stations_eu/"
OUTv10_PATH = "/data/flexpart/output/v10/flexpartweb/stations/"
OUTv11_PATH = "/data/flexpart/output/v11/flexpartweb/stations/"
OUTv10_EU_PATH = "/data/flexpart/output/v10/flexpartweb/stations_eu/"
OUTv11_EU_PATH = "/data/flexpart/output/v11/flexpartweb/stations_eu/"
sdate = '2025-08-01'
edate = '2025-08-20'

In [ ]:
# Merge footprint files for each station and domain
for StationID in lst_Station:
    # merge_footprint(FLEXPARTv10_PATH, StationID, sdate, edate, fname="foot", domain="global")
    # merge_footprint(FLEXPARTv11_PATH, StationID, sdate, edate, fname="foot", domain="global")
    # merge_footprint(FLEXPARTv10_PATH, StationID, sdate, edate, fname="foot_nest", domain="global_nest")
    # merge_footprint(FLEXPARTv11_PATH, StationID, sdate, edate, fname="foot_nest", domain="global_nest")
    # merge_footprint(FLEXPARTv10_EU_PATH, StationID, sdate, edate, fname="foot", domain="eu")
    # merge_footprint(FLEXPARTv11_EU_PATH, StationID, sdate, edate, fname="foot", domain="eu")
    # merge_footprint_out(FLEXPARTv10_PATH, OUTv10_PATH, StationID, sdate, edate, fname="foot", domain="global")
    # merge_footprint_out(FLEXPARTv11_PATH, OUTv11_PATH, StationID, sdate, edate, fname="foot", domain="global")
    # merge_footprint_out(FLEXPARTv10_PATH, OUTv10_PATH, StationID, sdate, edate, fname="foot_nest", domain="global_nest")
    # merge_footprint_out(FLEXPARTv11_PATH, OUTv11_PATH, StationID, sdate, edate, fname="foot_nest", domain="global_nest")
    # merge_footprint_out(FLEXPARTv10_EU_PATH, OUTv10_EU_PATH, StationID, sdate, edate, fname="foot", domain="eu")
    # merge_footprint_out(FLEXPARTv11_EU_PATH, OUTv11_EU_PATH, StationID, sdate, edate, fname="foot", domain="eu")
    merge_footprint_parallel(FLEXPARTv10_PATH, OUTv10_PATH, StationID, sdate, edate, fname="foot", domain="global")
    merge_footprint_parallel(FLEXPARTv11_PATH, OUTv11_PATH, StationID, sdate, edate, fname="foot", domain="global")
    merge_footprint_parallel(FLEXPARTv10_PATH, OUTv10_PATH, StationID, sdate, edate, fname="foot_nest", domain="global_nest")
    merge_footprint_parallel(FLEXPARTv11_PATH, OUTv11_PATH, StationID, sdate, edate, fname="foot_nest", domain="global_nest")
    merge_footprint_parallel(FLEXPARTv10_EU_PATH, OUTv10_EU_PATH, StationID, sdate, edate, fname="foot", domain="eu")
    merge_footprint_parallel(FLEXPARTv11_EU_PATH, OUTv11_EU_PATH, StationID, sdate, edate, fname="foot", domain="eu")


In [10]:
sdate = '2025-08-01'
edate = '2025-08-02'
StationID = "htm150"
merge_footprint_out(FLEXPARTv10_PATH, OUTv10_PATH, StationID, sdate, edate, fname="foot", domain="global")

HTM150: 

Saved /data/flexpart/output/v10/flexpartweb/stations/HTM150/foot_global_2025-08-01_2025-08-02_flexpart.nc
HTM150 took 1.68 seconds


In [9]:
sdate = '2025-08-01'
edate = '2025-08-10'
StationID = "htm150"
merge_footprint_out(FLEXPARTv10_PATH, OUTv10_PATH, StationID, sdate, edate, fname="foot", domain="global")


HTM150: 
2025-08-05 00:00:00...2025-08-10 00:00:00...
Saved /data/flexpart/output/v10/flexpartweb/stations/HTM150/foot_global_2025-08-01_2025-08-10_flexpart.nc
HTM150 took 211.04 seconds


In [ ]:
lst_Station = ["htm150", "ope120", "gat341", "lin098", "cbw200"]
FLEXPART_PATH = FLEXPARTv10_PATH  # or FLEXPARTv11_PATH, set as needed
sdate = '2025-08-01'
edate = '2025-08-20'
period = pd.date_range(start=sdate, end=edate, inclusive='both')

for StationID in lst_Station:
    i = 0
    darray_flexpart = None
    for date in period:
        Year = str(date.year)
        Month = str(date.month).zfill(2)
        Day = str(date.day).zfill(2)
        for Hour in range(24):
            floder_Flexpart = Year + "x" + Month + "x" + Day + "x" + str(Hour).zfill(2)
            file_path = FLEXPART_PATH + StationID + "/" + Year + "/" + Month + "/" + floder_Flexpart + "/foot_nest"
            try:
                darray_temp = xr.open_dataarray(file_path)
                darray_temp = darray_temp.where(darray_temp != 0)
                if darray_flexpart is None:
                    darray_flexpart = darray_temp
                else:
                    darray_flexpart = xr.concat([darray_flexpart, darray_temp], dim="time")
            except Exception as e:
                print(f"Missing or error in file: {file_path} -- {e}")
        i += 1
        if i % 10 == 0:
            print(f"{StationID}: {date}...", end="")
    if darray_flexpart is not None:
        file_path_name = FLEXPART_PATH + StationID + "/" + f"foot_eu_{sdate}_{edate}_flexpart.nc"
        # Remove file if exists
        if exists(file_path_name):
            remove(file_path_name)
        darray_flexpart.to_netcdf(file_path_name)
        print(f"\nSaved {file_path_name}")
    else:
        print(f"\nNo data found for {StationID} in period {sdate} to {edate}")
